In [108]:
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely
import folium
import math
import warnings
import pyproj


from shapely.prepared import prep
from shapely.geometry import Point,Polygon
from shapely.ops import transform,cascaded_union
from functools import partial
from geopy import distance
from scipy import spatial


warnings.filterwarnings('ignore')
%config Completer.use_jedi = False

In [109]:
# read and filter relevant shapefiles
file_path = '../data/district_shapefiles/District_Boundary.shp'
districts_gpd = gpd.read_file(file_path)
# khi_districts_list = ['KARACHI CENTRAL', 'KARACHI WEST', 'MALIR CANTONMENT',
#        'KORANGI CREEK CANTONMENT', 'MANORA CANTONMENT',
#        'CLIFTON CANTONMENT','KARACHI CANTONMENT', 'FAISAL CANTONMENT',
#        'KARACHI SOUTH','MALIR','KORANGI', 'KARACHI EAST']

khi_districts_list = ['KARACHI SOUTH','KARACHI EAST','CLIFTON CANTONMENT']
khi_districts_gpd = districts_gpd[districts_gpd.DISTRICT.isin(khi_districts_list)]
khi_districts_gpd = khi_districts_gpd.reset_index(drop=True)

In [110]:
khi_shape = cascaded_union(khi_districts_gpd.geometry)

In [134]:
loc = list(khi_districts_gpd.geometry[0].exterior.coords)[0]
loc = loc[::-1]
map_ =  folium.Map(location=loc,zoom_start=12)
map_.add_child(folium.GeoJson(data=khi_shape))
map_

In [112]:
# chaning projection form WGS84 to Web Mercator, this is done to 
poly = gpd.GeoDataFrame({'geometry':[khi_shape]},crs=4326).to_crs(epsg=3857)['geometry'].values[0]

In [114]:
latmin, lonmin, latmax, lonmax = poly.bounds

In [115]:
prep_polygon = prep(poly)
valid_points = []
points = []

#each point in mesh will be 500 meters apart
resolution = 500

for lat in np.arange(latmin, latmax, resolution):
    for lon in np.arange(lonmin, lonmax, resolution):
        points.append(Point((round(lat,4), round(lon,4))))

In [116]:
valid_points.extend(filter(prep_polygon.contains,points))

In [125]:
mesh_gpd = gpd.GeoDataFrame({'geometry':valid_points},crs=3857).to_crs(epsg=4326)
mesh_gpd = mesh_gpd.reset_index()


In [133]:
map_ =  folium.Map(location=loc,zoom_start=12)
for ind, row in mesh_gpd.iterrows():
    point = row.geometry
    lat, lon = point.y, point.x
    folium.CircleMarker([lat, lon], radius=0.1, color='blue', fill=True, fill_color='blue').add_to(map_)

map_

In [156]:
mesh_gpd['lat'] = mesh_gpd.geometry.y
mesh_gpd['lng'] = mesh_gpd.geometry.x

In [161]:
def func_latlon_from_lat_lon(row):
    return (round(row.lat,5),round(row.lng,5))
mesh_gpd['lat_lng'] = mesh_gpd.apply(func_latlon_from_lat_lon,1)

In [165]:
vor = spatial.Voronoi(list(mesh_gpd['lat_lng'].values))

In [181]:
def func_voronoi_finite_polygons_2d(vor, radius=None):
    """
    Reconstruct infinite voronoi regions in a 2D diagram to finite
    regions.
    Parameters
    ----------
    vor : Voronoi
        Input diagram
    radius : float, optional
        Distance to 'points at infinity'.
    Returns
    -------
    regions : list of tuples
        Indices of vertices in each revised Voronoi regions.
    vertices : list of tuples
        Coordinates for revised Voronoi vertices. Same as coordinates
        of input vertices, with 'points at infinity' appended to the
        end.
    """

    if vor.points.shape[1] != 2:
        raise ValueError("Requires 2D input")

    new_regions = []
    new_vertices = vor.vertices.tolist()

    center = vor.points.mean(axis=0)
    if radius is None:
        radius = vor.points.ptp().max()*2

    # Construct a map containing all ridges for a given point
    all_ridges = {}
    for (p1, p2), (v1, v2) in zip(vor.ridge_points, vor.ridge_vertices):
        all_ridges.setdefault(p1, []).append((p2, v1, v2))
        all_ridges.setdefault(p2, []).append((p1, v1, v2))

    # Reconstruct infinite regions
    for p1, region in enumerate(vor.point_region):
        vertices = vor.regions[region]

        if all(v >= 0 for v in vertices):
            # finite region
            new_regions.append(vertices)
            continue

        # reconstruct a non-finite region
        ridges = all_ridges[p1]
        new_region = [v for v in vertices if v >= 0]

        for p2, v1, v2 in ridges:
            if v2 < 0:
                v1, v2 = v2, v1
            if v1 >= 0:
                # finite ridge: already in the region
                continue

            # Compute the missing endpoint of an infinite ridge

            t = vor.points[p2] - vor.points[p1] # tangent
            t /= np.linalg.norm(t)
            n = np.array([-t[1], t[0]])  # normal

            midpoint = vor.points[[p1, p2]].mean(axis=0)
            direction = np.sign(np.dot(midpoint - center, n)) * n
            far_point = vor.vertices[v2] + direction * radius

            new_region.append(len(new_vertices))
            new_vertices.append(far_point.tolist())

        # sort region counterclockwise
        vs = np.asarray([new_vertices[v] for v in new_region])
        c = vs.mean(axis=0)
        angles = np.arctan2(vs[:,1] - c[1], vs[:,0] - c[0])
        new_region = np.array(new_region)[np.argsort(angles)]

        # finish
        new_regions.append(new_region.tolist())

    return new_regions, np.asarray(new_vertices)

def func_intersect_vor(row):
    return khi_shape.intersection(row.vor)


In [182]:
regions,vertices=func_voronoi_finite_polygons_2d(vor,radius=0.07)


In [189]:
khi_shape.wkt

'POLYGON ((67.07971876500005 24.827675923000072, 67.07953303400006 24.822695641000053, 67.08250736100007 24.819423882000024, 67.08399452400005 24.815854690000037, 67.08250736100007 24.81079833600006, 67.08042533200006 24.806039414000054, 67.08012790000004 24.80038819400005, 67.08131763000006 24.798008733000074, 67.08518425400007 24.795331839000028, 67.09073638900003 24.79214096100003, 67.09328460700004 24.791393280000023, 67.09472889100005 24.788504234000072, 67.09538316800007 24.787195463000046, 67.09902954100005 24.779901505000055, 67.10230255100004 24.77813530000003, 67.10727367300007 24.766348661000052, 67.11070009700006 24.761399382000036, 67.10742106700008 24.760283797000056, 67.08829020000007 24.74643533400007, 67.08343610000003 24.745864264000033, 67.08053200000006 24.748213000000078, 67.07935400000008 24.748178000000053, 67.07861000000008 24.748244000000057, 67.07769500000006 24.748469000000057, 67.07701000000003 24.748826000000065, 67.07686200000006 24.74869700000005, 67.0770

In [187]:
mesh_gpd

,index,geometry,lat,lng,lat_lng
0,0,POINT (66.97621 24.80704),24.807036,66.976212,"(24.80704, 66.97621)"
1,1,POINT (66.97621 24.81111),24.811113,66.976212,"(24.81111, 66.97621)"
2,2,POINT (66.97621 24.81519),24.815190,66.976212,"(24.81519, 66.97621)"
3,3,POINT (66.97621 24.81927),24.819267,66.976212,"(24.81927, 66.97621)"
4,4,POINT (66.97621 24.82334),24.823344,66.976212,"(24.82334, 66.97621)"
...,...,...,...,...,...
771,771,POINT (67.15138 24.92929),24.929289,67.151383,"(24.92929, 67.15138)"
772,772,POINT (67.15138 24.93336),24.933362,67.151383,"(24.93336, 67.15138)"
773,773,POINT (67.15138 24.93744),24.937435,67.151383,"(24.93744, 67.15138)"
774,774,POINT (67.15588 24.93336),24.933362,67.155875,"(24.93336, 67.15588)"


In [183]:
polys = []
for region in regions:
    poly = vertices[region]
    polys.append(Polygon(poly))

vor_gpd = gpd.GeoDataFrame(polys)
vor_gpd = vor_gpd.rename(columns={
    0:'vor'
})
vor_gpd['bounded_vor'] = vor_gpd.apply(func_intersect_vor,1)
vor_gpd = gpd.GeoDataFrame(vor_gpd,geometry='bounded_vor')
vor_gpd = vor_gpd.drop(columns='vor')
vor_gpd = vor_gpd.reset_index()
vor_gpd = vor_gpd.rename(columns={'index':'vor_id'})
vor_gpd['vor_id'] = vor_gpd.vor_id.apply(lambda x: 'vor'+str(x))

In [184]:
vor_gpd

,vor_id,bounded_vor
0,vor0,POLYGON EMPTY
1,vor1,POLYGON EMPTY
2,vor2,POLYGON EMPTY
3,vor3,POLYGON EMPTY
4,vor4,POLYGON EMPTY
...,...,...
771,vor771,POLYGON EMPTY
772,vor772,POLYGON EMPTY
773,vor773,POLYGON EMPTY
774,vor774,POLYGON EMPTY
